In [3]:
import pandas as pd
from tqdm.auto import tqdm

C:\My Files\VirtualENV\Half\Half_girlfriend\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv('../Data/Final_data.csv')

In [5]:
df

,ids,question,answer
0,0,Can we hang out tomorrow?,I'd love that! Let's make some plans.
1,1,Is there something you're not telling me?,"No, but sometimes I worry you might not trust me."
2,2,What did you dream about?,"I dreamed about us, it was beautiful."
3,3,Why don't you trust me?,"It's not that I don't trust you, I just need s..."
4,5,How was your day?,It was good! How about yours?
...,...,...,...
633,761,What's your biggest fear about discussing our ...,Making mistakes... but I trust us to learn and...
634,762,What's your favorite thing about how we balanc...,The way we prioritize our connection while res...
635,763,How do you handle differences in our sleep hab...,By finding compromises that allow us both to f...
636,764,Do you ever feel like you're not adventurous e...,Sometimes... but you appreciate the stability ...


In [6]:
# Here we have created a conbine text to create vectore of our dataframe

df['all_txt'] = "Question: " + df['question'] + " Answer: " + df['answer']

In [7]:
df['all_txt'][0]

"Question: Can we hang out tomorrow? Answer: I'd love that! Let's make some plans."

In [8]:
df

,ids,question,answer,all_txt
0,0,Can we hang out tomorrow?,I'd love that! Let's make some plans.,Question: Can we hang out tomorrow? Answer: I'...
1,1,Is there something you're not telling me?,"No, but sometimes I worry you might not trust me.",Question: Is there something you're not tellin...
2,2,What did you dream about?,"I dreamed about us, it was beautiful.",Question: What did you dream about? Answer: I ...
3,3,Why don't you trust me?,"It's not that I don't trust you, I just need s...",Question: Why don't you trust me? Answer: It's...
4,5,How was your day?,It was good! How about yours?,Question: How was your day? Answer: It was goo...
...,...,...,...,...
633,761,What's your biggest fear about discussing our ...,Making mistakes... but I trust us to learn and...,Question: What's your biggest fear about discu...
634,762,What's your favorite thing about how we balanc...,The way we prioritize our connection while res...,Question: What's your favorite thing about how...
635,763,How do you handle differences in our sleep hab...,By finding compromises that allow us both to f...,Question: How do you handle differences in our...
636,764,Do you ever feel like you're not adventurous e...,Sometimes... but you appreciate the stability ...,Question: Do you ever feel like you're not adv...


### Qdrant search RAG

In [9]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
from qdrant_client.http.models import VectorParams, Distance
import os

In [10]:
qclient = QdrantClient(path="../Data/Emb")

In [13]:
collectionName = "allEmb"

In [14]:
# Creating collection for mistal 
qclient.create_collection(
       collection_name=collectionName,
       vectors_config=VectorParams(size=4096, distance=Distance.COSINE),
   )
print(collectionName + " Collection created")

allEmb Collection created


In [15]:
# Here we create a function to create embedding using mistral7B

from ollama import Client

OLLAMA_HOST = "http://localhost:11434/"

ollama_client = Client(OLLAMA_HOST)

def get_embedding_mis(txt):
    embeddings = ollama_client.embeddings(model='mistral', prompt=txt)
    return embeddings['embedding']

In [16]:
shape = df.shape

In [17]:
not_app = 0
i = 0
for x in tqdm(range(shape[0])):
    # points = []
    # Get the text from the row
    # txt = row['Merge']
    try:
        txt = df['all_txt'][x]
        print(txt)
        result=get_embedding_mis(txt)
        #print(result)
        # category = texts[x].metadata['Category']
        ids = df['ids'][x]
        print(ids)

        qclient.upsert(
            collection_name=collectionName,
            wait=True,
            points=[
            PointStruct(
                id=x,
                vector=result,
                payload={"text": txt,"ids":str(ids)}
            )
            ]
        )
        i = i + 1
        #print("Inserted " + str(x)+ " document into Qdrant for Mistral")
    except Exception as e:
        print(f"This token is not append {x} and got exception {e}")
        not_app += 1

print(not_app)

  0%|                                                                                                   | 0/638 [00:00<?, ?it/s]

Question: Can we hang out tomorrow? Answer: I'd love that! Let's make some plans.


  0%|▏                                                                                        | 1/638 [00:05<1:02:55,  5.93s/it]

0
Question: Is there something you're not telling me? Answer: No, but sometimes I worry you might not trust me.


  0%|▎                                                                                          | 2/638 [00:10<52:35,  4.96s/it]

...

763
Question: Do you ever feel like you're not adventurous enough in trying new things in our relationship? Answer: Sometimes... but you appreciate the stability and comfort I bring to us.


100%|████████████████████████████████████████████████████████████████████████████████████████▊| 637/638 [40:29<00:05,  5.21s/it]

764
Question: How important is it to maintain a sense of playfulness and humor in our relationship? Answer: It's crucial... laughter and fun keep our bond strong through all of life's ups and downs.


100%|█████████████████████████████████████████████████████████████████████████████████████████| 638/638 [40:36<00:00,  3.82s/it]

765
0
